# EXP 113
## -33の目的変数をもつcard_idの分析

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

feat_no = '113_eda_'

### Tableau可視化用データセットの作成

In [14]:
import datetime
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

clf_pred = utils.read_pkl_gzip('../stack/1225_102_outlier_classify_lgb_auc0-88940189917734_83features.gz').set_index(key)
clf_pred.columns = [f"clf_{col}" for col in clf_pred.columns]
out_pred = utils.read_pkl_gzip('../stack/1225_172_en_route_lgb_CV1-55583888596708_83features.gz').set_index(key)
out_pred.columns = [f"out_{col}" for col in out_pred.columns]
best_pred = utils.read_pkl_gzip('../stack/1225_182_en_route_lgb_CV3-65604336032456_83features.gz').set_index(key)
best_pred.columns = [f"best_{col}" for col in best_pred.columns]

best_pred[clf_pred.columns] = clf_pred
best_pred[out_pred.columns] = out_pred

In [86]:
import glob

win_path = '../features/4_winner/*'
# Data Load
base = utils.read_df_pkl('../input/base*')
win_path_list = glob.glob(win_path)
train_path_list = []
test_path_list = []
for path in win_path_list:
    if path.count('train'):
        train_path_list.append(path)
    elif path.count('test'):
        test_path_list.append(path)

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
train_feature_list = utils.parallel_load_data(path_list=train_path_list)
test_feature_list = utils.parallel_load_data(path_list=test_path_list)
train = pd.concat(train_feature_list, axis=1)
train = pd.concat([base_train, train], axis=1)
test = pd.concat(test_feature_list, axis=1)
test = pd.concat([base_test, test], axis=1)

train_test = pd.concat([train, test], ignore_index=True)
train_test.head()

SystemExit: 

In [19]:
train_test.set_index(key, inplace=True)
best_pred = best_pred.join(train_test)
best_pred.to_csv(f'../output/{start_time[4:11]}_elo_exp113.csv', index=True)

### Tableau可視化結果から、outlierの二値分類による予測可否でグルーピングしたcard_idを取得

In [11]:
out_g = pd.read_csv('../input/elo_outlier_clf_pred_div.csv')
display(out_g.head())

df_out = train_test.merge(out_g, how='inner', on='card_id')
print(df_out.shape)

,div,card_id
0,0,C_ID_0a9a30162f
1,0,C_ID_0a33d755fc
2,0,C_ID_0ac55f7b51
3,0,C_ID_0aeccc1ef8
4,0,C_ID_0b45efc727


(2207, 86)


In [29]:
# corr = train_test.corr(method='pearson')
# corr['111_ker_hist_first_buy@']

In [22]:
out_corr = df_out.query("div==1").corr(method='pearson')
out_corr.to_csv('../output/1227_elo_corr_outlier_clfdiv1.csv')

In [20]:
df_out.query("div==0").to_csv('../output/1227_elo_eda_outlier_clfdiv.csv', index=False)

### historyデータ

In [4]:
hist = utils.read_df_pkl('../input/historical_transactions0*')
new = utils.read_df_pkl('../input/new*0*')
hist.head(10)

100%|██████████| 3/3 [00:00<00:00,  3.40it/s]


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37
5,Y,C_ID_4e6213e9bc,333,N,0,A,80,M_ID_50af771f8d,0,-0.734887,2018-02-24 08:45:05,1.0,9,37
6,Y,C_ID_4e6213e9bc,88,N,0,A,278,M_ID_5e8220e564,-11,-0.716855,2017-03-21 00:10:51,1.0,16,37
7,Y,C_ID_4e6213e9bc,3,N,0,A,80,M_ID_9d41786a50,-3,-0.657049,2017-11-18 20:05:55,1.0,16,37
8,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-8,-0.737967,2017-06-01 22:02:56,1.0,16,37
9,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_74ba14b5fc,-11,-0.715352,2017-03-16 15:41:22,1.0,16,37


In [30]:
d0 = df_out.query("div==0")
d1 = df_out.query("div==1")

In [33]:
hist0 = hist.loc[hist[key].isin(d0.card_id), :]
hist1 = hist.loc[hist[key].isin(d1.card_id), :]

In [35]:
hist0['purchase_date'].min()
hist0['purchase_date'].max()

2017-01-01 00:07:20
2018-02-28 23:00:24


In [24]:
print(hist['category_1'].dropna().drop_duplicates())
print(hist['category_2'].dropna().drop_duplicates())
print(hist['category_3'].dropna().drop_duplicates())

0      N
797    Y
Name: category_1, dtype: object
0      1.0
400    3.0
406    5.0
407    2.0
464    4.0
Name: category_2, dtype: float64
0      A
400    B
493    C
Name: category_3, dtype: object


In [26]:
train = utils.read_df_pkl('../input/train00*')
train.head()
print(train['feature_1'].drop_duplicates())
print(train['feature_2'].drop_duplicates())
print(train['feature_3'].drop_duplicates())

100%|██████████| 3/3 [00:00<00:00, 96.09it/s]

0    5
1    4
2    2
4    1
6    3
Name: feature_1, dtype: int64
0    2
1    1
3    3
Name: feature_2, dtype: int64
0    1
1    0
Name: feature_3, dtype: int64


### newデータ

In [42]:
new2017 = new.loc[new['purchase_date'].map(lambda x: True if str(x).count('2017') else False), :]
new2017.shape

(303483, 14)

In [47]:
hist_new2017 = hist.loc[hist[key].isin(new2017[key].values), :]
print(hist_new2017.shape)
hist_new2017.head(20)

(3110038, 14)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
2812,Y,C_ID_a9af70f52d,96,N,1,B,307,M_ID_02afa32a59,-1,-0.369742,2017-07-12 18:51:42,2.0,24,19
2813,Y,C_ID_a9af70f52d,96,N,1,B,705,M_ID_eaaa28db61,-3,-0.731926,2017-05-03 07:17:35,2.0,24,33
2814,Y,C_ID_a9af70f52d,96,N,1,B,705,M_ID_eaaa28db61,-1,-0.697380,2017-07-28 09:15:49,2.0,24,33
2815,Y,C_ID_a9af70f52d,96,N,1,B,360,M_ID_69934e0055,-4,-0.697320,2017-04-16 08:51:05,2.0,24,34
2816,Y,C_ID_a9af70f52d,96,N,1,B,705,M_ID_eaaa28db61,-2,-0.730093,2017-06-05 07:11:13,2.0,24,33
2817,Y,C_ID_a9af70f52d,96,N,1,B,367,M_ID_c715681500,0,-0.674781,2017-08-26 13:17:35,2.0,24,16
2818,Y,C_ID_a9af70f52d,117,N,1,B,307,M_ID_806656dfc9,-1,-0.506469,2017-07-24 11:17:05,4.0,13,19
2819,Y,C_ID_a9af70f52d,96,N,1,B,367,M_ID_c715681500,0,-0.704834,2017-08-28 16:09:24,2.0,24,16
2820,Y,C_ID_a9af70f52d,96,N,1,B,705,M_ID_eaaa28db61,-1,-0.718282,2017-07-30 11:10:46,2.0,24,33
2821,Y,C_ID_a9af70f52d,160,N,1,B,771,M_ID_3f48320bbd,0,-0.711130,2017-08-10 14:51:03,5.0,21,31


### newは各IDの直近2ヶ月を取り出したデータセット

In [55]:
hist_new2017[hist_new2017[key]=='C_ID_a9af70f52d'].sort_values(by='purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
2992,N,C_ID_a9af70f52d,117,N,1,B,307,M_ID_0e272cdbf2,-7,-0.493320,2017-01-01 10:26:05,4.0,13,19
2902,Y,C_ID_a9af70f52d,117,N,1,B,307,M_ID_0e272cdbf2,-7,-0.493320,2017-01-01 10:27:12,4.0,13,19
2958,Y,C_ID_a9af70f52d,117,N,1,B,705,M_ID_3c75ea64c5,-7,-0.704383,2017-01-01 12:35:57,4.0,13,33
2984,Y,C_ID_a9af70f52d,96,N,1,B,705,M_ID_eaaa28db61,-7,-0.709402,2017-01-02 10:44:22,2.0,24,33
2838,Y,C_ID_a9af70f52d,96,N,1,B,373,M_ID_6986e72b70,-7,-0.505732,2017-01-02 18:26:40,2.0,24,18
3009,Y,C_ID_a9af70f52d,96,N,1,B,420,M_ID_68fba10ff4,-7,-0.596643,2017-01-03 18:49:32,2.0,24,31
2904,Y,C_ID_a9af70f52d,96,N,1,B,40,M_ID_768caea057,-7,-0.721768,2017-01-04 11:06:08,2.0,24,21
2867,N,C_ID_a9af70f52d,96,N,1,B,705,M_ID_eaaa28db61,-7,-0.723692,2017-01-04 11:20:34,2.0,24,33
2930,N,C_ID_a9af70f52d,96,N,1,B,705,M_ID_eaaa28db61,-7,-0.723692,2017-01-04 11:22:22,2.0,24,33
2961,N,C_ID_a9af70f52d,96,N,1,B,705,M_ID_eaaa28db61,-7,-0.723692,2017-01-04 11:24:50,2.0,24,33


In [56]:
new2017[new2017[key]=='C_ID_a9af70f52d'].sort_values(by='purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1318765,Y,C_ID_a9af70f52d,96,N,1.0,B,307,M_ID_90ec2a0cc4,1,-0.693278,2017-09-02 13:15:32,2,24,19
1318767,Y,C_ID_a9af70f52d,96,N,1.0,B,813,M_ID_df0e39fea5,1,-0.446378,2017-09-04 10:28:19,2,24,29
1318766,Y,C_ID_a9af70f52d,96,N,1.0,B,68,M_ID_6178ef4cc2,1,-0.669085,2017-09-06 19:28:04,2,24,27
1318771,Y,C_ID_a9af70f52d,96,N,1.0,B,705,M_ID_67dd0ae188,1,-0.636223,2017-09-12 22:31:14,2,24,33
1318768,Y,C_ID_a9af70f52d,96,N,1.0,B,607,M_ID_b075fc7e19,1,-0.727749,2017-09-23 12:58:10,2,24,29
1318770,Y,C_ID_a9af70f52d,96,N,1.0,B,437,M_ID_a7ac860b05,2,-0.697320,2017-10-01 14:57:30,2,24,15
1318769,Y,C_ID_a9af70f52d,96,N,1.0,B,40,M_ID_ffc9c91792,2,-0.737892,2017-10-07 14:26:47,2,24,21


In [57]:
test = utils.read_df_pkl('../input/test0*')
test[test[key]=='C_ID_a9af70f52d']

100%|██████████| 3/3 [00:00<00:00, 80.90it/s]


,first_active_month,card_id,feature_1,feature_2,feature_3
60115,2016-12,C_ID_a9af70f52d,5,1,1


### new_min_yyyymmとfirst_active_monthのクロス集計

In [34]:
hist_max = hist.groupby(key)['purchase_date'].max()
new_min = new.groupby(key)['purchase_date'].min()
new_min.name = 'purchase_date_new_min'

hist_new_span = pd.concat([hist_max, new_min], axis=1)

from dateutil import parser
new_min = new_min.to_frame()
new_min['yyyymm'] = new_min['purchase_date_new_min'].map(lambda x:  str(x)[:7])

train = utils.read_df_pkl('../input/train0*')
test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([train, test], axis=0)

new_min_tt = new_min.join(train_test.set_index(key))

new_min_tt['value'] = 1
new_pivot = new_min_tt.pivot_table(index='yyyymm', columns='first_active_month', values='value', aggfunc='sum')
new_pivot.to_csv('../output/1227_elo_new_first_ym_first_active_pivot.csv')

### newのmin, maxとhistのmin, max purchasedateをtrain_testにつける

In [52]:
new_max_min = new.groupby(key)['purchase_date'].agg({'new_date_min':'min', 'new_date_max':'max'})
hist_max_min = hist.groupby(key)['purchase_date'].agg({'hist_date_min':'min', 'hist_date_max':'max'})

In [53]:
tt_date_max_min = train_test.set_index(key).join(new_max_min).join(hist_max_min)
tt_date_max_min.head(10)

,feature_1,feature_2,feature_3,first_active_month,target,new_date_min,new_date_max,hist_date_min,hist_date_max
card_id,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,2018-03-05 14:04:36,2018-04-29 11:23:05,2017-06-27 14:18:08,2018-02-25 09:31:15
C_ID_3d0044924f,4,1,0,2017-01,0.392913,2018-02-01 17:07:54,2018-03-30 06:48:26,2017-01-06 16:29:42,2018-01-31 22:31:09
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,2018-04-28 17:43:11,2018-04-28 17:43:11,2017-01-11 08:21:22,2018-02-27 19:08:25
C_ID_186d6a6901,4,3,0,2017-09,0.142495,2018-03-07 11:55:06,2018-04-18 11:00:11,2017-09-26 16:22:21,2018-02-28 11:44:40
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,2018-03-02 11:55:43,2018-04-28 18:50:25,2017-11-12 00:00:00,2018-02-28 20:40:41
C_ID_0894217f2f,4,2,0,2016-09,0.871585,2017-08-15 10:17:55,2017-09-15 21:06:17,2017-01-10 17:14:53,2017-07-29 15:37:18
C_ID_7e63323c00,3,2,1,2016-12,0.230129,2018-03-06 12:37:14,2018-04-06 17:52:25,2017-01-05 12:19:24,2018-02-23 16:09:32
C_ID_dfa21fc124,3,2,1,2017-09,2.135850,2018-02-06 07:03:41,2018-02-18 20:27:17,2017-09-28 21:24:46,2017-12-21 19:44:42
C_ID_fe0fdac8ea,2,1,0,2017-08,-0.065406,2018-03-17 14:22:31,2018-03-20 21:23:02,2017-08-18 16:21:38,2017-12-09 13:02:59


In [56]:
from tqdm import tqdm
date_cols = [col for col in tt_date_max_min.columns if col.count('date')]
for col in date_cols:
    tt_date_max_min[col] = tt_date_max_min[col].map(lambda x: pd.to_datetime(str(x)[:7]))

tt_date_max_min.head()

,feature_1,feature_2,feature_3,first_active_month,target,new_date_min,new_date_max,hist_date_min,hist_date_max
card_id,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,2018-03-01,2018-04-01,2017-06-01,2018-02-01
C_ID_3d0044924f,4,1,0,2017-01,0.392913,2018-02-01,2018-03-01,2017-01-01,2018-01-01
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,2018-04-01,2018-04-01,2017-01-01,2018-02-01
C_ID_186d6a6901,4,3,0,2017-09,0.142495,2018-03-01,2018-04-01,2017-09-01,2018-02-01
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,2018-03-01,2018-04-01,2017-11-01,2018-02-01


In [58]:
tt_date_max_min['term_days'] = (tt_date_max_min['new_date_max'] - tt_date_max_min['hist_date_min']).dt.days
tt_date_max_min['term_days_span_hist_new'] = (tt_date_max_min['new_date_min'] - tt_date_max_min['hist_date_max']).dt.days

for col in date_cols:
    tt_date_max_min[f"{col}_year"] = tt_date_max_min[col].map(lambda x: str(x)[:4])
tt_date_max_min.head()

,feature_1,feature_2,feature_3,first_active_month,target,new_date_min,new_date_max,hist_date_min,hist_date_max,term_days,term_days_span_hist_new,new_date_min_year,new_date_max_year,hist_date_min_year,hist_date_max_year
card_id,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,2018-03-01,2018-04-01,2017-06-01,2018-02-01,304.0,28.0,2018,2018,2017,2018
C_ID_3d0044924f,4,1,0,2017-01,0.392913,2018-02-01,2018-03-01,2017-01-01,2018-01-01,424.0,31.0,2018,2018,2017,2018
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,2018-04-01,2018-04-01,2017-01-01,2018-02-01,455.0,59.0,2018,2018,2017,2018
C_ID_186d6a6901,4,3,0,2017-09,0.142495,2018-03-01,2018-04-01,2017-09-01,2018-02-01,212.0,28.0,2018,2018,2017,2018
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,2018-03-01,2018-04-01,2017-11-01,2018-02-01,151.0,28.0,2018,2018,2017,2018


In [62]:
df = tt_date_max_min.copy()
year_cols = [col for col in df.columns if col.count('year')]
out = df[df[target]<-30]
for col in year_cols:
    display(out[col].value_counts())

2018    868
2017    765
NaT     574
Name: new_date_min_year, dtype: int64

2018    955
2017    678
NaT     574
Name: new_date_max_year, dtype: int64

2017    2207
Name: hist_date_min_year, dtype: int64

2017    1670
2018     537
Name: hist_date_max_year, dtype: int64

In [61]:
df['hist_date_min_year'].value_counts()

2017    325540
Name: hist_date_min_year, dtype: int64

In [75]:
out['span_year_new_hist'] = out['hist_date_min_year'].map(lambda x: int(x.replace('NaT', '2000'))) - out['new_date_max_year'].map(lambda x: int(x.replace('NaT', '2000')))
out['span_year_new_hist'].value_counts()

-1     955
 0     678
 17    574
Name: span_year_new_hist, dtype: int64

In [76]:
out[out['span_year_new_hist']==0]['hist_date_max_year'].value_counts()

2017    678
Name: hist_date_max_year, dtype: int64

In [71]:
out['new_date_max_year'].value_counts()

2018    955
2017    678
NaT     574
Name: new_date_max_year, dtype: int64

In [77]:
df['span_year_new_hist'] = df['hist_date_min_year'].map(lambda x: int(x.replace('NaT', '2000'))) - df['new_date_max_year'].map(lambda x: int(x.replace('NaT', '2000')))
df['span_year_new_hist'].value_counts()

-1     251415
 0      38586
 17     35539
Name: span_year_new_hist, dtype: int64

In [79]:
noout = df[df[target]>-30]
noout[noout['span_year_new_hist']==0]['hist_date_max_year'].value_counts()

2017    23418
Name: hist_date_max_year, dtype: int64

In [80]:
noout['new_date_max_year'].value_counts()

2018    154935
2017     23418
NaT      21357
Name: new_date_max_year, dtype: int64

In [85]:
df[~df[target].isnull()].to_csv('../output/1227_elo_eda_date_target.csv', index=False)